In [1]:
import numpy as np 
import pandas as pd 
import requests 
import math 
from scipy import stats 

In [2]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
df = pd.read_csv('S&P500.csv')
df.head()

,Symbol
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 86872873,
 'calculationPrice': 'close',
 'change': 3.48,
 'changePercent': 0.02507,
 'close': 142.03,
 'closeSource': 'oficilfa',
 'closeTime': 1717075020718,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 143.65,
 'delayedPriceTime': 1715429725390,
 'extendedChange': 0.08,
 'extendedChangePercent': 0.00058,
 'extendedPrice': 145.42,
 'extendedPriceTime': 1677868735859,
 'high': 144.89,
 'highSource': 'i 5ntaeeuedmcpel idy1 r',
 'highTime': 1724770319515,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 147.142,
 'iexCloseTime': 1738248951614,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 144.89,
 'iexOpenTime': 1701964385823,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1720513328310,
 'latestPrice': 147.25,
 'latestSource': 'Close',
 'latestTime': 'June 24, 2022',
 'latestUpdate': 1694251587081,
 'latestVolume': 89218744,
 'low': 140.

In [4]:
Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['peRatio'] = []

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
batches  = list(chunks(df['Symbol'], 100)) # 


for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['peRatio'].append(data[company]['quote']['peRatio'])
        

df = pd.DataFrame(Companies)
df

,Name,CMP,peRatio
0,A,123.39,29.40
1,AAL,14.00,-3.91
2,AAP,187.90,21.15
3,AAPL,146.15,24.02
4,ABBV,155.81,21.91
...,...,...,...
497,YUM,119.00,20.88
498,ZBH,113.71,108.82
499,ZBRA,314.19,20.82
500,ZION,54.32,5.56


In [5]:
df['peRatio'].fillna(value = df.peRatio.mean(), inplace = True)
lowPE = df.sort_values('peRatio')
lowPE = lowPE[lowPE['peRatio'] > 0]
lowPE = lowPE[0:50] # selecting top 50 based on P/E values
lowPE.reset_index(inplace = True)
lowPE

,index,Name,CMP,peRatio
0,335,NRG,39.37,2.43
1,152,EBAY,46.25,2.50
2,34,AMZN,118.76,2.80
3,339,NUE,115.02,4.00
4,424,SYF,29.97,4.12
5,175,F,12.11,4.33
6,314,MRNA,149.06,4.34
7,107,COF,113.00,4.39
8,383,PVH,65.52,4.60
9,18,AIG,53.30,4.74


In [6]:
portfolio_size = 100000
amount = portfolio_size/50

pd.set_option('mode.chained_assignment', None)
lowPE['Shares to buy'] = 0
for i in range(len(lowPE['Name'])):
    lowPE['Shares to buy'][i] = math.floor(amount/lowPE['CMP'][i])
    
lowPE

,index,Name,CMP,peRatio,Shares to buy
0,335,NRG,39.37,2.43,50
1,152,EBAY,46.25,2.50,43
2,34,AMZN,118.76,2.80,16
3,339,NUE,115.02,4.00,17
4,424,SYF,29.97,4.12,66
5,175,F,12.11,4.33,165
6,314,MRNA,149.06,4.34,13
7,107,COF,113.00,4.39,17
8,383,PVH,65.52,4.60,30
9,18,AIG,53.30,4.74,37


In [7]:
# Building a better model based on percentile of
# Price-to-earnings ratio - low(not always)
# Price-to-book ratio - low(within same sector)
# Price-to-sales ratio - low
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
# Enterprise Value divided by Gross Profit (EV/GP)

In [8]:
new_df = pd.read_csv('S&P500.csv')
new_df.head()

Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['peRatio'] = []
Companies['peRatio Percentile'] = []
Companies['pbRatio'] = []
Companies['pbRatio Percentile'] = []
Companies['psRatio'] = []
Companies['psRatio Percentile'] = []
Companies['EV/EBITDA'] = []
Companies['EV/EBITDA Percentile'] = []
Companies['EV/Profit'] = []
Companies['EV/Profit Percentile'] = []
Companies['Score'] = []


        
batches  = list(chunks(new_df['Symbol'], 100)) # 


for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['peRatio'].append(data[company]['quote']['peRatio'])
        Companies['peRatio Percentile'].append(0)
        Companies['pbRatio'].append(data[company]['advanced-stats']['priceToBook']
)
        Companies['pbRatio Percentile'].append(0)
        Companies['psRatio'].append(data[company]['advanced-stats']['priceToSales']
)
        Companies['psRatio Percentile'].append(0)
        if(data[company]['advanced-stats']['enterpriseValue'] == None or data[company]['advanced-stats']['EBITDA']==None):
            Companies['EV/EBITDA'].append(None)
        else:
            Companies['EV/EBITDA'].append(data[company]['advanced-stats']['enterpriseValue'] / data[company]['advanced-stats']['EBITDA'])
        Companies['EV/EBITDA Percentile'].append(0)
        
        if(data[company]['advanced-stats']['enterpriseValue'] == None or data[company]['advanced-stats']['grossProfit']==None):
            Companies['EV/Profit'].append(None)
        else:
            Companies['EV/Profit'].append(data[company]['advanced-stats']['enterpriseValue']/data[company]['advanced-stats']['grossProfit'])
              
        Companies['EV/Profit Percentile'].append(0)
        Companies['Score'].append(0)

    


new_df = pd.DataFrame(Companies)  

new_df

,Name,CMP,peRatio,peRatio Percentile,pbRatio,pbRatio Percentile,psRatio,psRatio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Profit,EV/Profit Percentile,Score
0,A,124.92,30.01,0,7.35,0,5.7600,0,20.593768,0,10.721271,0,0
1,AAL,14.20,-3.95,0,-1.03,0,0.2605,0,-21.914408,0,0.987405,0,0
2,AAP,183.90,20.47,0,3.93,0,1.0600,0,11.561596,0,2.511638,0,0
3,AAPL,146.14,23.92,0,35.41,0,6.1300,0,18.259202,0,14.508387,0,0
4,ABBV,158.33,22.82,0,16.60,0,4.9300,0,12.192338,0,8.529448,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,118.01,20.89,0,-4.00,0,5.0200,0,18.881937,0,8.547771,0,0
498,ZBH,112.15,110.71,0,2.00,0,3.1200,0,17.885033,0,5.563848,0,0
499,ZBRA,321.81,20.31,0,5.71,0,2.9400,0,15.737520,0,6.430095,0,0
500,ZION,55.22,5.75,0,1.40,0,2.4100,0,4.456772,0,2.146255,0,0


In [9]:
new_df['peRatio'].fillna(value = new_df.peRatio.mean(), inplace = True)  
new_df['pbRatio'].fillna(value = new_df.pbRatio.mean(), inplace = True)  
new_df['psRatio'].fillna(value = new_df.psRatio.mean(), inplace = True)  
new_df['EV/EBITDA'].fillna(value = new_df['EV/EBITDA'].mean(), inplace = True)  
new_df['EV/Profit'].fillna(value = new_df['EV/Profit'].mean(), inplace = True)

In [10]:
# calculating percentiles scores with the help of scipy
for i in range(len(new_df['Name'])):
    new_df['peRatio Percentile'][i] = stats.percentileofscore(new_df['peRatio'],new_df['peRatio'][i])/100
    new_df['pbRatio Percentile'][i] = stats.percentileofscore(new_df['pbRatio'],new_df['pbRatio'][i])/100
    new_df['psRatio Percentile'][i] = stats.percentileofscore(new_df['psRatio'],new_df['psRatio'][i])/100
    new_df['EV/EBITDA Percentile'][i] = stats.percentileofscore(new_df['EV/EBITDA'],new_df['EV/EBITDA'][i])/100
    new_df['EV/Profit Percentile'][i] = stats.percentileofscore(new_df['EV/Profit'],new_df['EV/Profit'][i])/100
    
    new_df['Score'][i] = (new_df['peRatio Percentile'][i]+new_df['pbRatio Percentile'][i]+new_df['psRatio Percentile'][i]+new_df['EV/EBITDA Percentile'][i]+new_df['EV/Profit Percentile'][i])/5


new_df = new_df.sort_values(by = 'Score')
new_df = new_df[:50]
new_df.reset_index(inplace = True)
new_df

,index,Name,CMP,peRatio,peRatio Percentile,pbRatio,pbRatio Percentile,psRatio,psRatio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Profit,EV/Profit Percentile,Score
0,1,AAL,14.20,-3.95,0.027888,-1.0300,0.063745,0.2605,0.011952,-21.914408,0.007968,0.987405,0.037849,0.029880
1,18,AIG,54.58,4.64,0.051793,0.7686,0.075697,0.7946,0.111554,2.814108,0.027888,0.749241,0.015936,0.056574
2,201,GM,35.59,6.07,0.085657,0.8293,0.081673,0.3932,0.029880,1.935579,0.017928,1.642210,0.087649,0.060558
3,383,PVH,64.89,4.71,0.053785,0.8291,0.079681,0.4685,0.039841,4.811836,0.087649,1.065677,0.049801,0.062151
4,207,GPS,9.58,-49.94,0.009960,1.4600,0.185259,0.2193,0.007968,5.000317,0.097610,0.726178,0.013944,0.062948
5,280,LNC,49.81,7.14,0.103586,0.5833,0.069721,0.4416,0.035857,5.362987,0.107570,0.439376,0.001992,0.063745
6,175,F,12.22,4.35,0.047809,1.0900,0.115538,0.3697,0.025896,3.818288,0.047809,1.933388,0.101594,0.067729
7,424,SYF,29.57,4.04,0.041833,1.2100,0.134462,0.9105,0.127490,2.474891,0.021912,0.930833,0.027888,0.070717
8,379,PRU,98.44,8.21,0.121514,0.8535,0.083665,0.5596,0.053785,4.915119,0.091633,0.528910,0.003984,0.070916
9,300,MET,66.41,8.38,0.131474,1.0028,0.107570,0.7422,0.095618,4.705030,0.079681,0.718598,0.011952,0.085259


In [11]:
portfolio_size = 100000
amount = portfolio_size/50

pd.set_option('mode.chained_assignment', None)
new_df['Shares to buy'] = 0
for i in range(len(new_df['Name'])):
    new_df['Shares to buy'][i] = math.floor(amount/new_df['CMP'][i])
    
new_df

,index,Name,CMP,peRatio,peRatio Percentile,pbRatio,pbRatio Percentile,psRatio,psRatio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Profit,EV/Profit Percentile,Score,Shares to buy
0,1,AAL,14.20,-3.95,0.027888,-1.0300,0.063745,0.2605,0.011952,-21.914408,0.007968,0.987405,0.037849,0.029880,140
1,18,AIG,54.58,4.64,0.051793,0.7686,0.075697,0.7946,0.111554,2.814108,0.027888,0.749241,0.015936,0.056574,36
2,201,GM,35.59,6.07,0.085657,0.8293,0.081673,0.3932,0.029880,1.935579,0.017928,1.642210,0.087649,0.060558,56
3,383,PVH,64.89,4.71,0.053785,0.8291,0.079681,0.4685,0.039841,4.811836,0.087649,1.065677,0.049801,0.062151,30
4,207,GPS,9.58,-49.94,0.009960,1.4600,0.185259,0.2193,0.007968,5.000317,0.097610,0.726178,0.013944,0.062948,208
5,280,LNC,49.81,7.14,0.103586,0.5833,0.069721,0.4416,0.035857,5.362987,0.107570,0.439376,0.001992,0.063745,40
6,175,F,12.22,4.35,0.047809,1.0900,0.115538,0.3697,0.025896,3.818288,0.047809,1.933388,0.101594,0.067729,163
7,424,SYF,29.57,4.04,0.041833,1.2100,0.134462,0.9105,0.127490,2.474891,0.021912,0.930833,0.027888,0.070717,67
8,379,PRU,98.44,8.21,0.121514,0.8535,0.083665,0.5596,0.053785,4.915119,0.091633,0.528910,0.003984,0.070916,20
9,300,MET,66.41,8.38,0.131474,1.0028,0.107570,0.7422,0.095618,4.705030,0.079681,0.718598,0.011952,0.085259,30
